# Downloading chart from last week

In [1]:
# Import libraries
import pandas as pd
import os
import re
import billboard
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langdetect import detect
pd.options.mode.chained_assignment = None

In [2]:
# Spotify authorization
spotify_client_id = os.environ.get('spotify_client_id')
spotify_client_secret = os.environ.get('spotify_client_secret')
client_credentials_manager = SpotifyClientCredentials(client_id = spotify_client_id, client_secret = spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
# Download data from last week
newest_chart = billboard.ChartData('hot-100')

In [4]:
# Creating Data Frame for data from last week
song_titles, artists, positions = [[] for i in range(3)]

position = 0
for i in newest_chart:
    position += 1
    positions.append(position)
    song = str(i).split(' by ')
    song_titles.append(song[0].strip("\'"))
    artists.append(song[1])

In [5]:
billboard_weekly_wrap_up = pd.DataFrame({'Position': positions, 'Artist': artists, 'Song Title': song_titles})
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title
0,1,Harry Styles,As It Was
1,2,Jack Harlow,First Class
2,3,Glass Animals,Heat Waves
3,4,Latto,Big Energy
4,5,Imagine Dragons X JID,Enemy
...,...,...,...
95,96,Gunna Featuring Drake,P Power
96,97,Yeat,Money So Big
97,98,Coi Leray & Nicki Minaj,Blick Blick!
98,99,Bailey Zimmerman,Fall In Love


In [6]:
# Spliting authors by 'and', 'featuring', 'feat.', Featuring', 'Feat.', '&' to make api searching easier and more efficient
artists = billboard_weekly_wrap_up['Artist'].to_list()
split_artists = []
for artist in artists:
    if any(re.findall(r'and|featuring|feat.|Featuring|Feat.|&|X', str(artist))):
        result = re.split(r'and|featuring|feat.|Featuring|Feat.|&|X', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

billboard_weekly_wrap_up['Split Names'] = split_artists
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names
0,1,Harry Styles,As It Was,Harry Styles
1,2,Jack Harlow,First Class,Jack Harlow
2,3,Glass Animals,Heat Waves,Glass Animals
3,4,Latto,Big Energy,Latto
4,5,Imagine Dragons X JID,Enemy,Imagine Dragons
...,...,...,...,...
95,96,Gunna Featuring Drake,P Power,Gunna
96,97,Yeat,Money So Big,Yeat
97,98,Coi Leray & Nicki Minaj,Blick Blick!,Coi Leray
98,99,Bailey Zimmerman,Fall In Love,Bailey Zimmerman


In [7]:
# Get Spotify ID
def get_spotify_ID(artist, track):
    sp_id_list = []
    music_genre_list = []
    audio_features = {}
    try:
        track_id = sp.search(q = 'artist:' + artist + ' track:' + track, type = 'track')
        sp_id = track_id['tracks']['items'][0]['id']
    except IndexError:
        sp_id = ''
    return sp_id
        
spotify_ids = []
for i in billboard_weekly_wrap_up.index:
    data = get_spotify_ID(billboard_weekly_wrap_up['Split Names'][i], billboard_weekly_wrap_up['Song Title'][i])
    spotify_ids.append(data)
    
billboard_weekly_wrap_up['Spotify ID'] = spotify_ids
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc
2,3,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp
3,4,Latto,Big Energy,Latto,6Zu3aw7FfjAF9WA0fA81Oq
4,5,Imagine Dragons X JID,Enemy,Imagine Dragons,1r9xUipOqoNwggBpENDsvJ
...,...,...,...,...,...
95,96,Gunna Featuring Drake,P Power,Gunna,0lEjxUUlKqjqXrVlIHFduD
96,97,Yeat,Money So Big,Yeat,7BbaIYAdi3pg4MGl6PHwPv
97,98,Coi Leray & Nicki Minaj,Blick Blick!,Coi Leray,7LczcBaamU9pTkV4Cl9NyX
98,99,Bailey Zimmerman,Fall In Love,Bailey Zimmerman,5gVCfYmQRPy1QJifP8f5gg


In [8]:
sum(billboard_weekly_wrap_up['Spotify ID'] == '')

2

In [9]:
billboard_weekly_wrap_up = billboard_weekly_wrap_up[billboard_weekly_wrap_up['Spotify ID'] != '']
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc
2,3,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp
3,4,Latto,Big Energy,Latto,6Zu3aw7FfjAF9WA0fA81Oq
4,5,Imagine Dragons X JID,Enemy,Imagine Dragons,1r9xUipOqoNwggBpENDsvJ
...,...,...,...,...,...
95,96,Gunna Featuring Drake,P Power,Gunna,0lEjxUUlKqjqXrVlIHFduD
96,97,Yeat,Money So Big,Yeat,7BbaIYAdi3pg4MGl6PHwPv
97,98,Coi Leray & Nicki Minaj,Blick Blick!,Coi Leray,7LczcBaamU9pTkV4Cl9NyX
98,99,Bailey Zimmerman,Fall In Love,Bailey Zimmerman,5gVCfYmQRPy1QJifP8f5gg


In [10]:
# Function returns audio features for specific Spotify ID
def get_audio_features(spotify_id):
    af = pd.DataFrame(sp.audio_features(spotify_id))
    return af

temp_audio_features_list = []
for i in billboard_weekly_wrap_up['Spotify ID']:
    temp_song_audio_features = get_audio_features(i)
    temp_audio_features_list.append(temp_song_audio_features)
temp_audio_features_df = pd.concat(temp_audio_features_list)
billboard_weekly_wrap_up = billboard_weekly_wrap_up.merge(temp_audio_features_df, how = 'inner', left_on = 'Spotify ID', right_on = 'id')
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
2,3,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp,0.761,0.525,11,-6.900,1,...,0.0921,0.531,80.870,audio_features,3USxtqRwSYz57Ewm6wWRMp,spotify:track:3USxtqRwSYz57Ewm6wWRMp,https://api.spotify.com/v1/tracks/3USxtqRwSYz5...,https://api.spotify.com/v1/audio-analysis/3USx...,238805,4
3,4,Latto,Big Energy,Latto,6Zu3aw7FfjAF9WA0fA81Oq,0.935,0.807,11,-3.838,0,...,0.3490,0.813,106.017,audio_features,6Zu3aw7FfjAF9WA0fA81Oq,spotify:track:6Zu3aw7FfjAF9WA0fA81Oq,https://api.spotify.com/v1/tracks/6Zu3aw7FfjAF...,https://api.spotify.com/v1/audio-analysis/6Zu3...,173182,4
4,5,Imagine Dragons X JID,Enemy,Imagine Dragons,1r9xUipOqoNwggBpENDsvJ,0.728,0.783,11,-4.424,0,...,0.4340,0.555,77.011,audio_features,1r9xUipOqoNwggBpENDsvJ,spotify:track:1r9xUipOqoNwggBpENDsvJ,https://api.spotify.com/v1/tracks/1r9xUipOqoNw...,https://api.spotify.com/v1/audio-analysis/1r9x...,173381,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,96,Gunna Featuring Drake,P Power,Gunna,0lEjxUUlKqjqXrVlIHFduD,0.828,0.591,1,-6.217,1,...,0.3610,0.202,119.251,audio_features,0lEjxUUlKqjqXrVlIHFduD,spotify:track:0lEjxUUlKqjqXrVlIHFduD,https://api.spotify.com/v1/tracks/0lEjxUUlKqjq...,https://api.spotify.com/v1/audio-analysis/0lEj...,193347,4
94,97,Yeat,Money So Big,Yeat,7BbaIYAdi3pg4MGl6PHwPv,0.693,0.575,2,-7.633,1,...,0.1040,0.353,138.016,audio_features,7BbaIYAdi3pg4MGl6PHwPv,spotify:track:7BbaIYAdi3pg4MGl6PHwPv,https://api.spotify.com/v1/tracks/7BbaIYAdi3pg...,https://api.spotify.com/v1/audio-analysis/7Bba...,160052,4
95,98,Coi Leray & Nicki Minaj,Blick Blick!,Coi Leray,7LczcBaamU9pTkV4Cl9NyX,0.916,0.824,8,-4.444,0,...,0.0661,0.870,139.926,audio_features,7LczcBaamU9pTkV4Cl9NyX,spotify:track:7LczcBaamU9pTkV4Cl9NyX,https://api.spotify.com/v1/tracks/7LczcBaamU9p...,https://api.spotify.com/v1/audio-analysis/7Lcz...,178413,4
96,99,Bailey Zimmerman,Fall In Love,Bailey Zimmerman,5gVCfYmQRPy1QJifP8f5gg,0.524,0.643,9,-6.055,1,...,0.2210,0.397,135.962,audio_features,5gVCfYmQRPy1QJifP8f5gg,spotify:track:5gVCfYmQRPy1QJifP8f5gg,https://api.spotify.com/v1/tracks/5gVCfYmQRPy1...,https://api.spotify.com/v1/audio-analysis/5gVC...,232059,4


In [11]:
wd = os.getcwd()
billboard_weekly_wrap_up.to_csv(wd + '\\last_week_chart.csv', index = False)